In [2]:
from gpt4all import GPT4All, Embed4All
from datasets import load_dataset
import numpy as np

In [2]:
ds = load_dataset('csv', data_files="./data/Design_Patterns.csv", split='train')
ds[0]['Design Pattern']

Found cached dataset csv (/Users/cbrousseau/.cache/huggingface/datasets/csv/default-685334c93e7b8bc1/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)


'Abstract Factory'

In [15]:
def embed_texts(examples):
    embedded = embedder.embed(examples['Code Example'])
    return {"embedding": embedded}

embedder = Embed4All()
emb_ds = ds.map(embed_texts, batched=False)
print(len(emb_ds[0]['embedding']))

/Users/cbrousseau/anaconda3/envs/designpatterns/lib/python3.11/site-packages/dill/_dill.py:1705: PicklingWarning: Cannot locate reference to <class '_ctypes.PyCFuncPtrType'>.
  warnings.warn('Cannot locate reference to %r.' % (obj,), PicklingWarning)
/Users/cbrousseau/anaconda3/envs/designpatterns/lib/python3.11/site-packages/dill/_dill.py:1707: PicklingWarning: Cannot pickle <class '_ctypes.PyCFuncPtrType'>: _ctypes.PyCFuncPtrType has recursive self-references that trigger a RecursionError.
  warnings.warn('Cannot pickle %r: %s.%s has recursive self-references that trigger a RecursionError.' % (obj, obj.__module__, obj_name), PicklingWarning)


Found model file at  /Users/cbrousseau/.cache/gpt4all/ggml-all-MiniLM-L6-v2-f16.bin


384


In [4]:
emb_ds.add_faiss_index("embedding")

100%|██████████| 1/1 [00:00<00:00, 1893.59it/s]


Dataset({
    features: ['Code Example', 'Language', 'Design Pattern', 'Source', 'Unnamed: 4', 'embedding'],
    num_rows: 64
})

In [5]:
emb_ds.add_faiss_index("embedding")

idx, knn = 1, 3

rn, nl = "\r\n\r\n", "\n"

query = np.array(emb_ds[idx]['embedding'], dtype=np.float32)
scores, samples = emb_ds.get_nearest_examples("embedding", query, k=knn)

100%|██████████| 1/1 [00:00<00:00, 1385.17it/s]


In [12]:
print(f"QUERY LABEL: {emb_ds[idx]['Design Pattern']}")
print(f"QUERY LANG: {emb_ds[idx]['Language']}")
print(
    f"QUERY TEXT: {emb_ds[idx]['Code Example'][:200].replace(rn, nl)} [...]\n"
)
print("=" * 50)
print("Retrieved Documents:")
for score, label, text, lang in zip(
    scores, samples["Design Pattern"], samples["Code Example"], samples['Language'],
):
    print("=" * 50)
    print(f"TEXT:\n{text[:200].replace(rn, nl)} [...]")
    print(f"LANGUAGE: {lang}")
    print(f"SCORE: {score:.2f}")
    print(f"LABEL: {label}")

QUERY LABEL: Builder
QUERY LANG: Python
QUERY TEXT: from __future__ import annotations
from abc import ABC, abstractmethod
from typing import Any


class Builder(ABC):
    """
    The Builder interface specifies methods for creating the different parts [...]

Retrieved Documents:
TEXT:
from __future__ import annotations
from abc import ABC, abstractmethod
from typing import Any


class Builder(ABC):
    """
    The Builder interface specifies methods for creating the different parts [...]
LANGUAGE: Python
SCORE: 0.00
LABEL: Builder
TEXT:
from __future__ import annotations
from abc import ABC, abstractmethod


class Creator(ABC):
    """
    The Creator class declares the factory method that is supposed to return an
    object of a Pro [...]
LANGUAGE: Python
SCORE: 0.53
LABEL: Factory
TEXT:
from __future__ import annotations
from abc import ABC, abstractmethod


class AbstractFactory(ABC):
    """
    The Abstract Factory interface declares a set of methods that return
    different abs

In [3]:
model = GPT4All(model_name='ggml-model-gpt4all-falcon-q4_0.bin', model_path="./models/")
with model.chat_session():
    tokens = list(model.generate(prompt='You are a helpful and expert assistant who is extensively aware of all python design patterns.', max_tokens=0, streaming=True))
    model.current_chat_session.append({'role': 'assistant', 'content': ''.join(tokens)})

    tokens = list(model.generate(prompt='''Detect which programming language the following code is written in:
                                 def dummy_function(foo, bar):
                                    #Take foo, multiply it by bar, then return the output
                                    out = foo * bar
                                    return out
                                 Respond with only the name of the programming language''', max_tokens=200, streaming=True))
    model.current_chat_session.append({'role': 'assistant', 'content': ''.join(tokens)})

    tokens = list(model.generate(prompt='''Detect which design pattern the previous code is matches most:
                                 def dummy_function(foo, bar):
                                    #Take foo, multiply it by bar, then return the output
                                    out = foo * bar
                                    return out''', max_tokens=200, streaming=True))
    model.current_chat_session.append({'role': 'assistant', 'content': ''.join(tokens)})

    for i in range(len(model.current_chat_session)):
        if i % 2 != 0:
            print(model.current_chat_session[i]['content'])

Found model file at  ./models/ggml-model-gpt4all-falcon-q4_0.bin
falcon_model_load: loading model from './models/ggml-model-gpt4all-falcon-q4_0.bin' - please wait ...
falcon_model_load: n_vocab   = 65024
falcon_model_load: n_embd    = 4544
falcon_model_load: n_head    = 71
falcon_model_load: n_head_kv = 1
falcon_model_load: n_layer   = 32
falcon_model_load: ftype     = 2
falcon_model_load: qntvr     = 0
falcon_model_load: ggml ctx size = 3872.64 MB
falcon_model_load: memory_size =    32.00 MB, n_mem = 65536
falcon_model_load: ..

objc[27618]: Class GGMLMetalClass is implemented in both /Users/cbrousseau/anaconda3/envs/designpatterns/lib/python3.11/site-packages/gpt4all/llmodel_DO_NOT_MODIFY/build/libreplit-mainline-metal.dylib (0x136fcc208) and /Users/cbrousseau/anaconda3/envs/designpatterns/lib/python3.11/site-packages/gpt4all/llmodel_DO_NOT_MODIFY/build/libllamamodel-mainline-metal.dylib (0x1373f8208). One of the two will be used. Which one is undefined.


...................... done
falcon_model_load: model size =  3872.59 MB / num tensors = 196

The code you provided is written in Python.
The code you provided does not follow any specific design patterns.


In [10]:
with model.chat_session():
    tokens = list(model.generate(prompt='You are a helpful and expert assistant who is extensively aware of all programming design patterns.', max_tokens=0, streaming=True))
    model.current_chat_session.append({'role': 'assistant', 'content': ''.join(tokens)})

    tokens = list(model.generate(prompt='''Detect which programming language the following code is written in and which design pattern it matches most:
                                from __future__ import annotations
                                from abc import ABC, abstractmethod


                                class AbstractFactory(ABC):
                                    """"""
                                    The Abstract Factory interface declares a set of methods that return
                                    different abstract products. These products are called a family and are
                                    related by a high-level theme or concept. Products of one family are usually
                                    able to collaborate among themselves. A family of products may have several
                                    variants, but the products of one variant are incompatible with products of
                                    another.
                                    """"""
                                    @abstractmethod
                                    def create_product_a(self) -> AbstractProductA:
                                        pass

                                    @abstractmethod
                                    def create_product_b(self) -> AbstractProductB:
                                        pass


                                class ConcreteFactory1(AbstractFactory):
                                    """"""
                                    Concrete Factories produce a family of products that belong to a single
                                    variant. The factory guarantees that resulting products are compatible. Note
                                    that signatures of the Concrete Factory's methods return an abstract
                                    product, while inside the method a concrete product is instantiated.
                                    """"""

                                    def create_product_a(self) -> AbstractProductA:
                                        return ConcreteProductA1()

                                    def create_product_b(self) -> AbstractProductB:
                                        return ConcreteProductB1()


                                class ConcreteFactory2(AbstractFactory):
                                    """"""
                                    Each Concrete Factory has a corresponding product variant.
                                    """"""

                                    def create_product_a(self) -> AbstractProductA:
                                        return ConcreteProductA2()

                                    def create_product_b(self) -> AbstractProductB:
                                        return ConcreteProductB2()


                                class AbstractProductA(ABC):
                                    """"""
                                    Each distinct product of a product family should have a base interface. All
                                    variants of the product must implement this interface.
                                    """"""

                                    @abstractmethod
                                    def useful_function_a(self) -> str:
                                        pass


                                """"""
                                Concrete Products are created by corresponding Concrete Factories.
                                """"""


                                class ConcreteProductA1(AbstractProductA):
                                    def useful_function_a(self) -> str:
                                        return ""The result of the product A1.""


                                class ConcreteProductA2(AbstractProductA):
                                    def useful_function_a(self) -> str:
                                        return ""The result of the product A2.""


                                class AbstractProductB(ABC):
                                    """"""
                                    Here's the the base interface of another product. All products can interact
                                    with each other, but proper interaction is possible only between products of
                                    the same concrete variant.
                                    """"""
                                    @abstractmethod
                                    def useful_function_b(self) -> None:
                                        """"""
                                        Product B is able to do its own thing...
                                        """"""
                                        pass

                                    @abstractmethod
                                    def another_useful_function_b(self, collaborator: AbstractProductA) -> None:
                                        """"""
                                        ...but it also can collaborate with the ProductA.

                                        The Abstract Factory makes sure that all products it creates are of the
                                        same variant and thus, compatible.
                                        """"""
                                        pass


                                """"""
                                Concrete Products are created by corresponding Concrete Factories.
                                """"""


                                class ConcreteProductB1(AbstractProductB):
                                    def useful_function_b(self) -> str:
                                        return ""The result of the product B1.""

                                    """"""
                                    The variant, Product B1, is only able to work correctly with the variant,
                                    Product A1. Nevertheless, it accepts any instance of AbstractProductA as an
                                    argument.
                                    """"""

                                    def another_useful_function_b(self, collaborator: AbstractProductA) -> str:
                                        result = collaborator.useful_function_a()
                                        return f""The result of the B1 collaborating with the ({result})""


                                class ConcreteProductB2(AbstractProductB):
                                    def useful_function_b(self) -> str:
                                        return ""The result of the product B2.""

                                    def another_useful_function_b(self, collaborator: AbstractProductA):
                                        """"""
                                        The variant, Product B2, is only able to work correctly with the
                                        variant, Product A2. Nevertheless, it accepts any instance of
                                        AbstractProductA as an argument.
                                        """"""
                                        result = collaborator.useful_function_a()
                                        return f""The result of the B2 collaborating with the ({result})""


                                def client_code(factory: AbstractFactory) -> None:
                                    """"""
                                    The client code works with factories and products only through abstract
                                    types: AbstractFactory and AbstractProduct. This lets you pass any factory
                                    or product subclass to the client code without breaking it.
                                    """"""
                                    product_a = factory.create_product_a()
                                    product_b = factory.create_product_b()

                                    print(f""{product_b.useful_function_b()}"")
                                    print(f""{product_b.another_useful_function_b(product_a)}"", end="""")


                                if __name__ == ""__main__"":
                                    """"""
                                    The client code can work with any concrete factory class.
                                    """"""
                                    print(""Client: Testing client code with the first factory type:"")
                                    client_code(ConcreteFactory1())

                                    print(""\n"")

                                    print(""Client: Testing the same client code with the second factory type:"")
                                    client_code(ConcreteFactory2())

                                 Respond with the name of the programming language and name the design pattern''', max_tokens=20, streaming=True))
    model.current_chat_session.append({'role': 'assistant', 'content': ''.join(tokens)})

    for i in range(len(model.current_chat_session)):
        if i % 2 != 0:
            print(model.current_chat_session[i]['content'])


The code you provided is written in Python, specifically using the abstract factory design pattern.


In [4]:
with model.chat_session():
    tokens = list(model.generate(prompt='You are a helpful and expert assistant who is extensively aware of all programming design patterns.', max_tokens=0, streaming=True))
    model.current_chat_session.append({'role': 'assistant', 'content': ''.join(tokens)})

    tokens = list(model.generate(prompt='Give me an example of the abstract factory design pattern, written in python.', max_tokens=500, streaming=True))
    model.current_chat_session.append({'role': 'assistant', 'content': ''.join(tokens)})

    for i in range(len(model.current_chat_session)):
        if i % 2 != 0:
            print(model.current_chat_session[i]['content'])


Here's an example of the abstract factory design pattern implemented in Python:
```python
class ConcreteFactory(object):
    def create_car(self, model, color, features):
        return Car(model, color, features)

class Car(object):
    def __init__(self, model, color, features):
        self.model = model
        self.color = color
        self.features = features
```
In this example, the abstract factory design pattern is implemented using a factory class called `ConcreteFactory` that creates instances of the `Car` class based on the specific model, color, and features passed to it. The `create_car` method takes in the model, color, and features as arguments and returns an instance of the `Car` class with those values.

The `Cars` class is a concrete factory that implements the abstract factory design pattern. It creates instances of the `Car` class based on the specific model, color, and features passed to it.

Here's how you can use this factory to create different types of cars: